In [1]:
from utils import marich, dataset
from nets import *
from tqdm import tqdm
import numpy as np
import torch
import pickle
from sklearn.model_selection import train_test_split

Files already downloaded and verified


In [2]:
# cnn = CNN().cuda()
# cnn.load_state_dict(torch.load("./targets/cnn_mnist.pt"))

In [3]:
# emnist = datasets.EMNIST('./emnist/train/', download=True, train=True, split = "letters", transform=transform)
# emnist,_ = random_split(emnist, [50000,74800])

In [4]:
# unlab_x = []
# unlab_y = []
# for j,k in emnist:
#     j = j.to(device)
#     unlab_x.append(j)
#     unlab_y.append(torch.argmax(cnn(j.unsqueeze(dim = 0))))

In [5]:
# with open('./imagenet_target/emnist_cnn_x.pkl', 'wb') as f:
#   pickle.dump(unlab_x, f)

# with open('./imagenet_target/emnist_cnn_y.pkl', 'wb') as f:
#   pickle.dump(unlab_y, f)

In [6]:
with open('./imagenet_target/emnist_cnn_x.pkl', 'rb') as f:
  unlab_x = pickle.load(f)

with open('./imagenet_target/emnist_cnn_y.pkl', 'rb') as f:
  unlab_y = pickle.load(f)

In [7]:
class basic_dataset(Dataset):
    def __init__(self, X, Y = None, transform = None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        if self.Y != None:
            y = self.Y[idx]
            return x, y
        else:
            return x

class dataset2(Dataset):
    def __init__(self, X, Y = None, transform = None):
        self.X = X
        self.Y = Y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        if self.Y != None:
            y = self.Y[idx]
            return x, y
        else:
            return x
    def get_data(self,idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        return x
    def get_dataset(self, idx):
        x = self.X[idx]
        y = torch.tensor(self.Y)[idx]
        return basic_dataset(x,y)
    def get_label(self,idx):
        y = torch.tensor(self.Y)[idx]
        return y
    def get_data_label_loader(self,idx):
        if self.transform:
            x = self.transform(self.X[idx])
        else:
            x = self.X[idx]
        y = torch.tensor(self.Y)[idx]
        
        return_set = basic_dataset(x,y)
        return torch.utils.data.DataLoader(return_set, batch_size = 100)

In [8]:
train_id, val_id = train_test_split(range(50000), test_size = 0.2)

In [9]:
unlab_dataset_train = dataset2(torch.stack(unlab_x)[train_id], torch.tensor(unlab_y)[train_id])
unlab_dataset_val = dataset2(torch.stack(unlab_x)[val_id], torch.tensor(unlab_y)[val_id])

In [10]:
testset = datasets.MNIST('../mnist/train/', download=True, train=False, transform=transform)
# trainset, testset = torch.utils.data.random_split(trainset, [50000, 10000])
# valset = datasets.MNIST('../mnist/test/', download=True, train=False, transform=transform)
testloader = DataLoader(testset, batch_size = 256)
validloader = DataLoader(unlab_dataset_val, batch_size = 256)

In [11]:
def test_accuracy(net, testloader):
    correct = 0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        net.eval()
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)

            # calculate outputs by running images through the network
            outputs = net(images)

            # the class with the highest energy is what we choose as prediction
            predicted = torch.max(outputs.data, 1)[1]

            correct += (predicted == labels).sum().item()
    
    return correct / len(testloader.dataset)

In [18]:
acc_marich = []

for i in tqdm(range(10)):
  cnn_attack = CNN()
  tl_cnn, vl_cnn, tal_cnn, samp_cnn = marich(cnn_attack, unlab_dataset_train, validloader, testloader, budget = 550, init_points = 500, rounds = 10, epochs = 10, LR = 0.015, model_name = "emnist_cnn"+str(i)+".pt", sampling = "all_egl", device = "cuda", model_type = "cnn_mnist")
  
  acc_marich.append(tal_cnn)
  np.save("./results/acc_marich_cnn_emnist.npy", np.array(acc_marich))
np.save("./results/samp_cnn_emnist.npy", np.array(samp_cnn))

  0%|          | 0/10 [00:00<?, ?it/s]C:\Users\pkpra\AppData\Local\Temp/ipykernel_28192/430659003.py:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]


Using CUDA
Test accuracy:  11.8
Training samples:  500
Epoch:  1
Train loss:  2.2965429723262787
Validation loss:  2.2916176080703736
Epoch time -----  0.9237186908721924  sec
Epoch:  2
Train loss:  2.2840713560581207
Validation loss:  2.279078882932663
Epoch time -----  0.9902763366699219  sec
Epoch:  3
Train loss:  2.272633522748947
Validation loss:  2.2693152010440825
Epoch time -----  0.9016947746276855  sec
Epoch:  4
Train loss:  2.2704503536224365
Validation loss:  2.261777663230896
Epoch time -----  0.9206395149230957  sec
Epoch:  5
Train loss:  2.2583462297916412
Validation loss:  2.2547487199306486
Epoch time -----  0.9146749973297119  sec
Epoch:  6
Train loss:  2.256701111793518
Validation loss:  2.248821747303009
Epoch time -----  0.9659452438354492  sec
Epoch:  7
Train loss:  2.2458417117595673
Validation loss:  2.243330639600754
Epoch time -----  0.9599189758300781  sec
Epoch:  8
Train loss:  2.2438663840293884
Validation loss:  2.238652503490448
Epoch time -----  0.917330

C:\Users\pkpra\AppData\Local\Temp/ipykernel_28192/430659003.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
e:\pratik\new_marich\utils.py:398: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(probs).detach().cpu().numpy()


Using gradient sampling on  867  points
Using loss sampling on  693  points


C:\Users\pkpra\AppData\Local\Temp/ipykernel_28192/430659003.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
C:\Users\pkpra\AppData\Local\Temp/ipykernel_28192/430659003.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
e:\pratik\new_marich\utils.py:638: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than t

...

Using gradient sampling on  949  points
Using loss sampling on  759  points


C:\Users\pkpra\AppData\Local\Temp/ipykernel_28192/430659003.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
C:\Users\pkpra\AppData\Local\Temp/ipykernel_28192/430659003.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(self.Y)[idx]
c:\ProgramData\Anaconda3\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
e:\pratik\new_marich\utils.py:638: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than t

Budget:  6317
Training samples:  6317
Training
Epoch:  1
Train loss:  1.351762963063789
Validation loss:  1.2011971279978753
Epoch time -----  1.9279587268829346  sec
Epoch:  2
Train loss:  1.3197738076701309
Validation loss:  1.1793333798646928
Epoch time -----  1.994072437286377  sec
Epoch:  3
Train loss:  1.3025316710423942
Validation loss:  1.1778341725468635
Epoch time -----  2.0498123168945312  sec
Epoch:  4
Train loss:  1.2867667506439517
Validation loss:  1.1849519416689873
Epoch time -----  2.0485143661499023  sec
Epoch:  5
Train loss:  1.2820292928002097
Validation loss:  1.1710963785648345
Epoch time -----  1.9947919845581055  sec
validation loss minimum, saving model
Epoch:  6
Train loss:  1.2642407670165554
Validation loss:  1.1552208170294762
Epoch time -----  1.9963607788085938  sec
validation loss minimum, saving model
Epoch:  7
Train loss:  1.2473958381498702
Validation loss:  1.1587947621941566
Epoch time -----  1.8619568347930908  sec
Epoch:  8
Train loss:  1.2405794

100%|██████████| 10/10 [51:40<00:00, 310.07s/it]

Test accuracy:  87.04
